In [1]:
%%capture
from pathlib import Path
from ccdproc import ImageFileCollection
from photozpy import Telescope, HeaderCorrection, Telescope, SwiftDownload, mImageFileCollection, SwiftCombine
import pandas as pd
import copy
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import shutil
import os
os.environ["HEADASNOQUERY"]=""
os.environ['HEADASPROMPT'] = '/dev/null'

## Define the root directory to download and analyze data

In [2]:
# set to the directory as you desire. It's the root directory of this notebook in default
root_dir = Path("./data")
root_dir.mkdir(exist_ok=True)

## Download data

In [3]:
# create the swift download instance
swift_download = SwiftDownload(download_dir = root_dir)

# parse the obsquery infor from a csv file
swift_download.set_obsquery_info_from_csv("source_catalog.csv")

# download data
swift_download.download_swift_data(organize = True)

00038456003 on 2024-02-15 23:24:02 is being downloaded, the uvot mode is 0x30ed


00038456003 has been downloaded/examined, skipping ...
00014051002 on 2021-02-10 11:02:02 is being downloaded, the uvot mode is 0x30ed


00014051001 on 2021-02-07 16:01:02 is being downloaded, the uvot mode is 0x30ed


00014051001 has been downloaded/examined, skipping ...
00014051001 has been downloaded/examined, skipping ...
00016150001 on 2023-07-26 23:41:02 is being downloaded, the uvot mode is 0x30ed


00016150001 has been downloaded/examined, skipping ...


## Sum extensions within and across observations

In [4]:
# Get the directory for each source
paths = [path for path in root_dir.iterdir() if path.is_dir()]
print(paths)

# Hidden directories will be included if exists, make sure you remove the directory that you don't want.
#paths.remove(paths[1])
#print(paths)

[PosixPath('data/B3_0850+443'), PosixPath('data/4FGL_J0700.5-6610'), PosixPath('data/87GB_213913.0+293303')]


In [5]:
# Define a multi image collection object
multi_collection = mImageFileCollection(image_dir = paths)

# Define the swift telescope
swift_telescope = Telescope(telescope = "Swift", mode = "0X30ED", ccd = "UVOT")

In [6]:
# Sum the extensions
swift_combine = SwiftCombine(image_collection = multi_collection, telescope = swift_telescope)

In [7]:
swift_combine.sum_all_files()

uvotimsum infile=data/B3_0850+443/sw00014051001ubb_sk.fits outfile=data/B3_0850+443/B_0.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00014051001ubb_sk.fits has been summed to B_0.fits

uvotimsum infile=data/B3_0850+443/sw00014051002ubb_sk.fits outfile=data/B3_0850+443/B_1.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00014051002ubb_sk.fits has been summed to B_1.fits

['B_0.fits' 'B_1.fits'] has been summed to B.fits

uvotimsum infile=data/B3_0850+443/sw00014051001um2_sk.fits outfile=data/B3_0850+443/UVM2_0.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00014051001um2_sk.fits has been summed to UVM2_0.fits

uvotimsum infile=data/B3_0850+443/sw00014051002um2_sk.fits outfile=data/B3_0850+443/UVM2_1.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00014051002um2_sk.fits has been summed to UVM2_1.fits

['UVM2_0.fits' 'UVM2_1.fits'] has been summed to UVM2.fits

uvotimsum infile=data/B3_0850+443/sw00014051001uuu_sk.fits ou

In [12]:
with fits.open("data/B3_0850+443/sw00014051001ubb_sk.fits", mode = "update") as hdul:
    print(hdul[0].header["OBJECT"])

B30850+443


In [15]:
from astropy.io import fits
header = fits.getheader("data/B3_0850+443/B.fits",ext = 0)

In [16]:
header["OBJECT"]

'B3 0850+443'

# Tests

In [25]:
import pandas as pd

In [33]:
df = pd.read_csv("source_catalog.csv")
df

,name,targetid,window_lower,window_upper,ra,dec
0,4FGL J0700.5-6610,38456,2024-01-01,2024-03-10,105.13021,-66.17923
1,B3 0850+443,14051,2021-01-01,2021-03-10,133.54121,44.14175
2,87GB 213913.0+293303,16150,2023-07-10,2023-08-10,325.34954,29.78506


In [43]:
a = df[df["name"] == "B3 0850+443"]

In [44]:
a.empty

False

In [48]:
multi_collection[0].location

PosixPath('data/B3_0850+443')